### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from pydicom import dcmread
from pydicom.data import get_testdata_file
from PIL import Image

from tensorflow.keras import Sequential, layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.layers.experimental.preprocessing import Rescaling ?

## Création du dataset

### Import dataset créé à partir des fichiers .csv

In [ ]:
from covid_project.data import merged_df, get_simple_df
csv_df = merged_df()
simple_df = get_simple_df()
print(csv_df.head(3))
simple_df.head(3)

### Création dataset image

Utilisation d'un code OU de l'autre, les codes se recoupent

#### Code Claire

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [3]:
#Zeinab google drive path
gdrive_path = 'https://drive.google.com/drive/folders/1kTQVjjV71w08EGUHnoVV1MCwdP7PxR0r' #à modifier
dcm_path = os.path.join('..','raw_data')

In [12]:
def load_image_data(loading_method):
    if loading_method == 'colab':
        data_path = gdrive_path
    elif loading_method == 'direct':
        data_path = dcm_path
    file_names = [f for f in os.listdir(data_path) if f.endswith('.dcm')]
    image_id = [key_name.replace('.dcm','') for key_name in file_names]
    image_data = []
    for ids,file in zip(image_id, file_names):
        ds = dcmread(os.path.join(data_path, file))
        image_data.append([ids, ds.StudyInstanceUID, ds.PatientSex, ds.ImagerPixelSpacing, ds.PhotometricInterpretation, ds.Rows, ds.Columns, ds.pixel_array])
    return image_data

In [13]:
image_data = load_image_data('direct')
image_data[0]

['65761e66de9f',
 '00086460a852',
 'F',
 2330,
 2783,
 array([[1392, 1389, 1390, ...,    0,    0,    0],
        [1377, 1411, 1370, ...,    0,    0,    0],
        [1373, 1397, 1401, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ..., 2476, 2490, 2453],
        [   0,    0,    0, ..., 2515, 2541, 2509],
        [   0,    0,    0, ..., 2517, 2489, 2488]], dtype=uint16)]

In [15]:
train_metadata = pd.DataFrame(image_data, columns=['image_id','image_StudyInstanceUID','Patient_Sex', "ImagerPixelSpacing", "PhotometricInterpretation", 'image_Rows','image_Columns','pixel_array'])
print(images_df.shape)
train_metadata.head()

(5, 6)


,image_id,image_StudyInstanceUID,Patient_Sex,image_Rows,image_Columns,Pixel_array
0,65761e66de9f,00086460a852,F,2330,2783,"[[1392, 1389, 1390, 1390, 1350, 1385, 1360, 13..."
1,51759b5579bc,000c9c05fd14,M,3093,2850,"[[82, 81, 80, 81, 79, 80, 81, 78, 79, 77, 76, ..."
2,bb4b1da810f3,0051d9b12e72,F,2336,2836,"[[24335, 24448, 24548, 24551, 24336, 24244, 24..."
3,3019399c31f4,005057b3f880,F,3052,3052,"[[2848, 2863, 2885, 2891, 2899, 2919, 2894, 28..."
4,f6293b1c49e2,00292f8c37bd,M,2539,3050,"[[86, 88, 86, 86, 87, 86, 87, 87, 87, 87, 87, ..."


#### Code Zeinab

In [ ]:
destination_path =  "/mnt/c/Users/Administrator/Desktop/COVID_Data/train_subset/New_train_subset" 

In [ ]:
images_list = os.listdir(destination_path)

In [ ]:
columns = ["PatientSex", "ImagerPixelSpacing",
           "SeriesInstanceUID", "PhotometricInterpretation",
           "Rows", "Columns", "pixel_array"]

In [ ]:
train_metadata =  pd.DataFrame(columns=columns)
new_images_list =[]
for i in images_list:
    fs=pydicom.read_file(destination_path+"/"+i, stop_before_pixels=True)
    train_metadata = train_metadata.append({field: fs[field].value for field in columns}, ignore_index=True)
    element = i.replace(".dcm", "")
    new_images_list.append(element)
train_metadata['image_id'] = new_images_list

In [ ]:
print(train_metadata.shape)
train_metadata.head()

### Merge des df

In [ ]:
big_df = simple_df.merge(train_metadata, on = 'image_id', how='inner')
bif_df.head()

In [ ]:
X = big_df['pixel_array'] # ??? à vérifier, à passer en np.array ou en matrice? garder plus d'infos dans le df ?
y = big_df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']]

In [ ]:
print(type(X))
print(X.shape)
len(X)

In [ ]:
#train_val_test_split

from sklearn.model_selection import train_test_split

def train_val_test_split(X,y):
    X_bis, X_test, y_bis, y_test = train_test_split(X, y, test_size=0.3)
    X_train, X_val, y_train, y_val = train_test_split(X_bis, y_bis, test_size=0.2)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X,y)

## Modèle CNN